### Convert tex to qmd

In [8]:
texchapters = [f"texfiles/Chapter{i}.tex" for i in range(1, 15)] + ["texfiles/AppendixB.tex"]
qmdchapters = [f"Chapter{i}.qmd" for i in range(1, 15)] + ["AppendixB.qmd"]

for cread, cwrite in zip(texchapters, qmdchapters):
    
    with open(cread, 'r') as f:
        contents = f.read()

    contents = contents.replace("It\^o", "Ito")
    contents = contents.replace("= $", "=$")
    contents = contents.replace("\\D", "\\mathrm{d}")
    contents = contents.replace("\\E", "\\mathrm{e}")
    contents = contents.replace("\\varDelta", "\\Delta")
    contents = contents.replace("\\var(", "\\mathrm{var}(")
    contents = contents.replace("\\cov(", "\\mathrm{cov}(")
    contents = contents.replace("\\M", "\\mathrm{M}")
    contents = contents.replace("\\begin{equation}", "$$")
    contents = contents.replace("\\end{equation}", "$$")
    contents = contents.replace("\\N", "\\mathrm{N}")
    contents = contents.replace("\\begin{petit}", "::: {.callout-note}\n ## \n")
    contents = contents.replace("\\end{petit}", ":::\n")
    contents = contents.replace("\\citet", "\\cite")
    contents = contents.replace("\\citet*", "\\cite")
    contents = contents.replace("\\renewcommand{\\labelenumi}{(\\alph{enumi})}", "")
    contents = contents.replace("\\section*{Problems}", "\n## Exercises\n")
    contents = contents.replace("\\addcontentsline{toc}{section}{Problems}", "")
    contents = contents.replace("\\notag", "")
    contents = contents.replace("\\newpage", "")
    contents = contents.replace("\\normalsize", "")
    contents = contents.replace("\\large", "")
    contents = contents.replace("\\small", "")
    contents = contents.replace("\\scriptsize", "")
    contents = contents.replace("\\footnotesize", "")
    contents = contents.replace("\\normalsize", "")

    contents = contents.replace("\\n(", "\\mathrm{n}(")
    contents = contents.replace("\\n ", "\\mathrm{n} ")

    # fix Prob
    go = True
    while go:
        try:
            i = contents.index("Prob.~\\ref{")
            j = contents.index("}", i)
            tag = contents[(i+11):j]
            contents = contents[:i] + "@exr-" + tag + contents[j+1:]
        except Exception as e:
            go = False
    
    # fix textbf
    go = True
    while go:
        try:
            i = contents.index("\\textbf{")
            j = contents.index("}", i)
            text = contents[i+8:j]
            contents = contents[:i] + "**" + text + "**" + contents[j+1:]
        except:
            go = False

    # fix textit
    go = True
    while go:
        try:
            i = contents.index("\\textit{")
            j = contents.index("}", i)
            text = contents[i+8:j]
            contents = contents[:i] + "**" + text + "**" + contents[j+1:]
        except:
            go = False

    # fix multline
    go = True
    s = 0
    while go:
        try:
            i = contents.index("\\begin{multline}", s)
            j = contents.index("\\end{multline}", i)
            go2 = True
            while go2:
                try:
                    k = contents.index("\\label{", i, j)
                    m = contents.index("}", k)
                    tag = contents[k+7:m]
                    contents = contents[:k] + contents[m+1:]
                except:
                    go2 = False
            j = contents.index("\\end{multline}", i)
            contents = contents[:j+14] + "\n$$ {#eq-" + tag + "}\n" + contents[j+14:]
            contents = contents[:i] + "\n$$\n" + contents[i:]
            s = i + 5
        except:
            go = False

    # fix align
    go = True
    while go:
        try:
            i = contents.index("\\begin{align}")
            j = contents.index("\\end{align}", i)
            go2 = True
            while go2:
                try:
                    k = contents.index("&", i, j)
                    go3 = True
                    while go3:
                        try:
                            m = contents.index("\intertext{", i, j)
                            n = contents.index("}", m)
                            tag = contents[m+11:n]
                            contents = contents[:m] + "$$\n" + tag + "\n$$" + contents[n+1:]
                        except:
                            go3 = False
                    go3 = True
                    while go3:
                        try:
                            m = contents.index("\\\\", k, j)
                            contents = contents[:m] + "\n$$\n$$\n" + contents[m+2:]
                        except:
                            go3 = False
                    contents = contents[:k] + contents[k+1:]
                except:
                    go2 = False 
            j = contents.index("\\end{align}", i)
            contents = contents[:j] + "$$\n" + contents[j+11:]
            contents = contents[:i] + "\n$$" + contents[i+13:]
        except:
            go = False




    # fix subequations
    go = True
    while go:
        try:
            i = contents.index("\\begin{subequations}\\label{")
            j = contents.index("}", i+26)
            contents = contents[:i] + contents[j+1:]
        except:
            go = False

    contents = contents.replace("\\begin{subequations}", "")
    contents = contents.replace("\\end{subequations}", "")

    go = True
    while go:
        try:
            i = contents.index("\\begin{subequations} \\label{")
            j = contents.index("}", i+27)
            contents = contents[:i] + contents[j+1:]
        except:
            go = False

    contents = contents.replace("\\begin{subequations}", "")
    contents = contents.replace("\\end{subequations}", "")

    # fix exercises
    go = True
    while go:
        try:
            i = contents.index("\\begin{prob}")
            j = contents.index("\\end{prob}", i)
            try:
                k = contents.index("\\label{", i, j)
                m = contents.index("}", k)
                tag = contents[k+7: m]
            except:
                tag = None
            contents = contents[:j] + ":::\n" + contents[j+10:]
            if tag:
                contents = contents[:k] + contents[m+1:]
                contents = contents[:i] + "::: {#exr-" + tag + "}\n" + contents[i+12:]
            else:
                contents = contents[:i] + "::: {#exr-nolabel}\n" + contents[i+12:]
        except:
            go = False

 
    # fix cite
    go = True
    while go:
        try:
            i = contents.index("\\cite{")
            j = contents.index("}", i)
            tag = contents[i+6:j]
            contents = contents[:i] + "[@" + tag + "]" + contents[j+1:]
        except Exception as e:
            go = False
        
    # fix citep
    go = True
    while go:
        try:
            i = contents.index("\\citep{")
            j = contents.index("}", i)
            tag = contents[i+7:j]
            contents = contents.index[:i] + "([@" + tag + "])" + contents[j+1:]
        except Exception as e:
            go = False
            
    # fix eqref
    go = True
    while go:
        try:
            i = contents.index("\\eqref{")
            j = contents.index("}", i)
            tag = contents[i+7:j]
            contents = contents[:i] + "@eq-" + tag + contents[j+1:]
        except Exception as e:
            go = False

    # fix footnotes
    go = True
    while go:
        try:
            i = contents.index("\\footnote{")
            j = contents.index("}", i)
            note = contents[i+10: j]
            contents = contents[:i] + "^[" + note + "]" + contents[j+1:]
        except Exception as e:
            go = False 
          
    # fix quotation marks
    go = True
    while go:
        try:
            i = contents.index("``")
            j = contents.index("''", i)
            contents = contents[:j] + contents[j+2:]
            contents = contents[:i] + contents[i+2:]
        except Exception as e:
            go = False
  
    # fix tag
    go = True 
    while go:
        try:
            i = contents.index("\\tag{")
            j = contents.index("}", i)
            j = contents.index("}", j+1)
            contents = contents[:i] + contents[j+1:]
        except:
            go = False
  
    # fix chapter title
    try:
        i = contents.index("\\chapter{")
        j = contents.index("}", i)
        title = contents[i+9:j]
        k = contents.index("\label{", i)
        m = contents.index("}", k)
        tag = contents[k+7:m]
        contents = "# " + title + " {#sec-" + tag + contents[m:]
    except Exception as e:
        pass
    
    # fix mybox
    go = True
    while go:
        try:
            i = contents.index("\\mybox{")
            contents = contents[:i] + "\n::: {.callout-tip}\n## \n" + contents[i+7:]
        except Exception as e:
            go = False 
    
    # fix }
    go = True
    left = []
    right = []
    cursor = 0
    while go:
        try:
            i = contents.index("{", cursor)
            left.append(i)
            cursor = i + 1
        except Exception as e:
            go = False 
    
    cursor = 0
    go = True
    while go:
        try:
            i = contents.index("}", cursor)
            right.append(i)
            cursor = i + 1
        except Exception as e:
            go = False 

    if len(left) < len(right):
        extras = []
        while len(left) > 0:
            i = left.pop(0)
            j = right.pop(0)
            while (j < i) and (len(right)>len(left)):
                extras.append(j)
                j = right.pop(0)
 
        while len(extras) > 0:
            j = extras.pop(-1)
            contents = contents[:j] + "\n:::\n" + contents[j+1:]
    
    """
    # fix intertext
    go = True
    while go:
        try:
            i = contents.index("\intertext{")
            j = contents.index("}", i)
            tag = contents[i+11:j]
            contents = contents[:i] + "$$\n" + tag + "\n$$" + contents[j+1:]
        except Exception as e:
            go = False
    """

    # fix itemize
    contents = contents.replace("\\bi ", "\\begin{itemize}")
    contents = contents.replace("\\ei", "\\end{itemize}")
    contents = contents.replace("\\im", "\\item")
    go = True
    while go:
        try:
            i = contents.index("\\begin{itemize}")
            j = contents.index("\\end{itemize}", i)
            k = contents.index("\\item", i)
            while k < j:
                contents = contents[:k] + "-" + contents[k+5:]
                j = contents.index("\\end{itemize}", i)
                try:
                    k = contents.index("\\item", i)
                except Exception as e:
                    k = j + 1
            contents = contents[:j] + "\n" + contents[j+13:]
            contents = contents[:i] + "\n" + contents[i+15:]
        except Exception as e:
            go = False
 
    # fix enumerate
    go = True
    while go:
        try:
            i = contents.index("\\begin{enumerate}")
            j = contents.index("\\end{enumerate}", i)
            k = contents.index("\\item", i)
            num = 1
            while k < j:
                contents = contents[:k] + str(num) + "." + contents[k+5:]
                num += 1
                j = contents.index("\\end{enumerate}", i)
                try:
                    k = contents.index("\\item", i)
                except Exception as e:
                    k = j + 1
            contents = contents[:j] + "\n" + contents[j+15:]
            contents = contents[:i] + "\n" + contents[i+17:]
        except Exception as e:
            go = False

    strs = ["equation ", "equation~", "equation  ", "equations ", "equations~"]
    strs = [s + "\\eqref" for s in strs]
    for s in strs:
        contents = contents.replace(s, "\\eqref")

    # fix Sect
    go = True
    while go:
        try:
            i = contents.index("Sect")
            j = contents.index("}", i)
            tag = contents[(i+11):j]
            contents = contents[:i] + "@sec-" + tag + contents[j+1:]
        except Exception as e:
            go = False

    # fix Sects
    go = True
    while go:
        try:
            i = contents.index("Sects.~\\ref{")
            j = contents.index("}", i)
            tag = contents[(i+11):j]
            contents = contents[:i] + "@sec-" + tag + contents[j+1:]
        except Exception as e:
            go = False
 
    # fix Chap
    go = True
    while go:
        try:
            i = contents.index("Chap.~\\ref{")
            j = contents.index("}", i)
            tag = contents[(i+11):j]
            contents = contents[:i] + "@sec-" + tag + contents[j+1:]
        except Exception as e:
            go = False

    # fix section
    go = True
    while go:
        try:
            i = contents.index("\\section{")
            contents = contents[:i] + "## " + contents[i+9:]
            j = contents.index("}", i)
            contents = contents[:j] + contents[j+1:]
            k = contents.index("\\label{", i-1)
            if k<=j+2:
                contents = contents[:k] + " {#sec-" + contents[k+7:]
        except Exception as e:
            go = False
    
    # fix chapter
    go = True
    while go:
        try:
            i = contents.index("\\chapter{")
            contents = contents[:i] + "## " + contents[i+9:]
            j = contents.index("}", i)
            contents = contents[:j] + contents[j+1:]
            k = contents.index("\\label{", i-1)
            if k<=j+2:
                contents = contents[:k] + " {#sec-" + contents[k+7:]
        except Exception as e:
            go = False

    # fix chapter[
    go = True
    while go:
        try:
            i = contents.index("\\chapter[")
            j = contents.index("]{", i)
            k = contents.index("}", j)
            m = contents.index("\\label{", k)
            n = contents.index("}", m)
            name = contents[j+2:k]
            tag = contents[m+7:n]
            if m <= k + 3:
                contents = contents[:m] + " {#sec-" + tag + "}" + contents[n+1:]
            contents = contents[:i] + "## " + name + contents[k+1:]
        except Exception as e:
            go = False

    

    # fix subsection        
    go = True
    while go:
        try:
            i = contents.index("\\subsection*{")
            j = contents.index("}", i)
            contents = contents[:j] + contents[j+1:]
            contents = contents[:i] + "### " + contents[i+13:]
        except Exception as e:
            go = False
    

    """
    # fix & 
    go = True
    i = 0
    while go:
        try: 
            i = contents.index("&", i)
            print("i", i)
            try:
                j = contents.index("\\text{", i, i+10)
                print("j", j)
                i = i + 1
            except:
                contents = contents[:i] + contents[i+1:]
        except:
            go = False

    # fix \\
    go = True
    i = 0
    while go:
        try: 
            i = contents.index("\\\\", i)
            try:
                j = contents.index("\\text{", i, i+15)
                i = i + 1
            except:
                contents = contents[:i] + "\n$$\n$$" + contents[i+2:]
        except:
            go = False   
   
    contents = contents.replace("\\notag", "")
    """ 

    # fix equation labels
    go = True
    while go:
        try:
            i = contents.index("\\label{")
            j = contents.index("}", i)
            tag = contents[i+7:j]
            k = contents.index("$$", i)
            contents = contents[:k] + "$$ {#eq-" + tag + "}\n" + contents[k+2:]
            contents = contents[:i] + contents[j+1:]
        except Exception as e:
            go = False
    
    contents = contents.replace("sec-s_continuouscompounding", "sec-c_basics")


    contents = contents.replace("$$\n\n", "$$\n")
    contents = contents.replace("{eq-", "{#eq-")
    with open(cwrite, 'w') as f:
        f.write(contents)

### Fix references to subequations in qmd files

In [17]:
qmdchapters = [f"Chapter{i}.qmd" for i in range(1, 15)] + ["AppendixB.qmd"]

for chapter in qmdchapters:
    
    with open(chapter, 'r') as f:
        contents = f.read()

    contents = contents.replace("~$", " $")
    contents = contents.replace("equation @", "@")
    contents = contents.replace("the @", "@")
    contents = contents.replace("formula @", "@")
    contents = contents.replace("definition @", "@")

    contents = contents.replace("@eq-binomialSR10combined", "@eq-binomialS10 - @eq-binomialR10")
    contents = contents.replace(
        "@eq-itoprocess110111",
        "@eq-itoprocess110 - @eq-itoprocess111"
    )
    contents = contents.replace(
        "@eq-heston12combined",
        "@eq-heston1 - @eq-heston2"
    )
    contents = contents.replace(
        "@eq-blackcombinedfutures",
        "@eq-black2 - @eq-black_d22"
    )
    contents = contents.replace(
        "@eq-blackcombined",
        "@eq-black - @eq-black_d2"
    )
    
    contents = contents.replace(
        "@eq-Mertoncombined",
        "@eq-merton - @eq-merton_d2again"
    )
    contents = contents.replace(
        "@eq-forwardstartcombined",
        "@eq-forwardstart_d1 - @eq-forwardstart_d2"
    )
    contents = contents.replace(
        "@eq-americancallcombined",
        "@eq-americancalld1 - @eq-americancalld2"
    )
    contents = contents.replace(
        "@eq-americancallstarcombined",
        "@eq-americancalld1star - @eq-americancalld2star"
    )
    contents = contents.replace(
        "@eq-max12combined",
        "@eq-max1 - @eq-max2"
    )
    contents = contents.replace(
        "@eq-max34combined",
        "@eq-max4 - @eq-max3"
    )
    contents = contents.replace(
        "@eq-casea441combined",
        "@eq-casea4 - @eq-casea41"
    )
    contents = contents.replace(
        "@eq-casea35combined",
        "@eq-casea3 - @eq-casea5"
    )
    contents = contents.replace(
        "@eq-caseb35combined",
        "@eq-caseb3 - @ceq-aseb5"
    )
    contents = contents.replace(
        "@eq-fslc100combined",
        "@eq-fslc100a - @eq-fslc100c"
    )
    contents = contents.replace(
        "@eq-prob13combined",
        "@eq-prob1 - @eq-prob3"
    )
    contents = contents.replace(
        "@eq-a13combined",
        "@eq-a1 - @eq-a3"
    )
    contents = contents.replace(
        "@eq-mc_twoabcombined",
        "@eq-mc_twoa - @eq-mc_twob"
    )
    contents = contents.replace(
        "@eq-crank4aabbcombined",
        "@eq-crank4aa - @eq-crank4bb"
    )
    contents = contents.replace(
        "@eq-vasicekcapletfloorlet",
        "@eq-vasicekfloorlet - @eq-vasicekcaplet"
    )

    contents = contents.replace(
        "@eq-vasicekbondcombined",
        "@eq-vasicekbond - @eq-vasicekb1111"
    )
    contents = contents.replace(
        "@eq-captions13combined",
        "@eq-captions1 - @eq-captions3"
    )
    contents = contents.replace(
        "@eq-vasicekextension_solcombined",
        "@eq-vasicekextension_sol - @eq-rhat103"
    )
    contents = contents.replace(
        "@eq-vasicekdiscountcallput",
        "@eq-vasicekdiscountcall - @eq-vasicekdiscountcall_d2"
    )
    contents = contents.replace(
        "@eq-extendedvasicekbondcombined",
        "@eq-extendedvasicekbond - @eq-extendedvasicek_b"
    )
    contents = contents.replace(
        "@eq-cirbondcombined",
        "@eq-cirbeta - @eq-ciralpha"
    )
    contents = contents.replace(
        "@eq-x12lscombined",
        "@eq-x1ls - @eq-x2ls"
    )
    with open(chapter, 'w') as f:
        f.write(contents)

In [7]:
tags = "@forwardstart_d2 @americalcalld2  @americalcalld2star @max2 @max3 @casea41 @casea5 @caseb5 @fslc100c".split()
for chapter in ["Chapter8.qmd"]:
    
    with open(chapter, 'r') as f:
        contents = f.read()

    for tag in tags:
        contents = contents.replace(tag, f"@eq-{tag[1:]}")

    with open(chapter, 'w') as f:
        f.write(contents)

### Convert bititems to bibfile

In [5]:
with open("texfiles/References.tex", "r") as f:
    lines = f.readlines()

file = ""

for line in lines[1:-1]:
    if "broadiekaya" in line or "CV" in line or "JW" in line or "Stulz" in line:
        pass
    else:
        item = "@ARTICLE{" if "textbf" in line else "@BOOK{"
        i = line.index("{")
        j = line.index("}", i)
        tag = line[i+1: j]
        item += tag + ",\n author={"
        k = line.index(":", j)
        names = line[j+1: k].split(",")
        author = ""
        while len(names) > 0:
            first = names.pop(0)
            last = names.pop(0)
            author += first + "," + last + " and "
        author = author[:-5]
    
        if "textbf" in line:
            item += author + "},\n title={"
            m = line.index(".", k)
            title = line[k+1: m]
            item += title + "},\n journal={" 
            n = line.index(",", m)
            journal = line[m+1: n]
            item += journal +"},\n volume={"
            i = line.index("\\textbf{", n)
            j = line.index("}", i)
            volume = line[i+8:j]
            item += volume + "},\n pages={"
            k = line.index("(", j)
            pages = line[j+3: k]
            item += pages + "},\n year={"
            m = line.index(")", k)
            year = line[k+1: m]

        elif tag not in ("broadiekaya", "CV"):
            item += author + "},\n title={"
            m = line.index(",", k)
            title = line[k+1: m]
            item += title + "},\n publisher={"
            n = line.index(",", m)
            publisher = line[m+1: n]
            item += publisher +"},\n address={"
            i = line.index("(", n)
            address = line[n+1: i-1]
            item += address +"},\n year={"
            m = line.index(")", i)
            year = line[i+1: m]

        item += year +"}\n}"
        file += item + "\n"


            
with open("references.bib", 'w') as f:
    f.write(file)
